<a href="https://colab.research.google.com/github/Ayanlola2002/DATA-SCIENCE-PROJECTS/blob/master/Hotel_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Importing needed libraries
import numpy as np
import pandas as pd
from ast import literal_eval
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [9]:
df=pd.read_csv('/content/drive/MyDrive/Hotel_reviews/Hotel_review_feature_engineering.csv')


In [10]:
df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Country,City,tourist,Trip_type,Travelling_Status,stay_duration,room_small,wi_fi,air_conditioning,breakfast,booking_com,room_problem,location,staff,bed_and_room,month,year
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.700,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.900,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916,Netherlands,Amsterdam,Tourist,1.000,2,6,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,8,2017
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.700,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.500,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916,Netherlands,Amsterdam,Tourist,1.000,2,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,8,2017
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.700,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.100,"[' Leisure trip ', ' Family with young childre...",3 days,52.361,4.916,Netherlands,Amsterdam,Tourist,1.000,4,3,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,7,2017
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.700,Hotel Arena,UK,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.800,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.361,4.916,Netherlands,Amsterdam,Tourist,1.000,1,3,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,7,2017
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.700,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.700,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.361,4.916,Netherlands,Amsterdam,Tourist,1.000,2,2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,7,2017


In [11]:
# Dropping unneeded columns
df.drop(['Unnamed: 0', 'Additional_Number_of_Scoring',
       'Review_Date','Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng', 'City', 'tourist',
       'Trip_type', 'Travelling_Status', 'stay_duration', 'room_small',
       'wi_fi', 'air_conditioning', 'breakfast', 'booking_com', 'room_problem',
       'location', 'staff', 'bed_and_room', 'month', 'year'],1,inplace=True)

In [12]:

df.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,Country
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.700,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.700,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.700,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.700,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.700,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",Netherlands


In [ ]:
class Tags_Country_recommender():
    def __init__(self,data=df):
        self.data = data
        # Seperating the tags components to make it clearer
        self.data['Tags'] = self.data['Tags'].apply(lambda x:"".join(literal_eval(x)))
        # Making these columns lowercase
        self.data['Country']=self.data['Country'].str.lower()
        self.data['Tags']=self.data['Tags'].str.lower()
      
    def recommend(self,location,description):   
        # Dividing the texts into small tokens (sentences into words)
        description = description.lower()
        description_tokens=word_tokenize(description)  
        sw = stopwords.words('english')  # List of predefined english stopwords to be used for computing
        lemm = WordNetLemmatizer() # This groups similar words so that it can be analyzed as a single word
        
        # We now define the functions below connecting these imported packages
        filtered_sen = {w for w in description_tokens if not w in sw}
        f_set=set()
        for fs in filtered_sen:
            f_set.add(lemm.lemmatize(fs))
        
        
        # Defining a new variable that takes in the location inputted and bring out the features defined below
        country_feat = self.data[self.data['Country']==location.lower()]
        country_feat = country_feat.set_index(np.arange(country_feat.shape[0]))
        cos=[]
        for i in range(country_feat.shape[0]):
            country_tokens=word_tokenize(country_feat['Tags'][i])
            filtered_set={w for w in country_tokens if not w in sw}
            t_set=set()
            for te in filtered_set:
                t_set.add(lemm.lemmatize(te))
            rvector = t_set.intersection(f_set)
            cos.append(len(rvector))
        country_feat['similarity']=cos
        country_feat=country_feat.sort_values(by='similarity',ascending=False)
        country_feat.drop_duplicates(subset='Hotel_Name',keep='first',inplace=True)
        country_feat.sort_values('Average_Score',ascending=False,inplace=True)
        country_feat.reset_index(inplace=True)
        # Printing top 10 recommendations based on the country and descriptions given
        # Prints out both the hotel name and its location
        for i in range(10):
            print (f'We recommend {country_feat.iloc[i,3]} located at {country_feat.iloc[i,1]}')


In [ ]:

recommender = Tags_Country_recommender()

In [ ]:
recommender.recommend('Netherlands','I am going on a business trip, I need a standard room and i am staying for two nights ')